In [ ]:
# -*- coding: utf-8
# Abraji (https://www.abraji.org.br)
# Reinaldo Chaves (reinaldo@abraji.org.br)
# Acessa as emendas da reforma trabalhista, faz o download dos PDFs e
# extrai os metadados que mostram os autores dos documentos
#

Gerou esta reportagem: https://theintercept.com/2017/04/26/lobistas-de-bancos-industrias-e-transportes-quem-esta-por-tras-das-emendas-da-reforma-trabalhista/

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import urllib.request, urllib.parse, urllib.error
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
import sys
import chardet
import csv
import pandas as pd

In [2]:
# Lista de links com os PDFs

In [3]:
emendas = urlopen("http://www.camara.gov.br/proposicoesWeb/prop_emendas?idProposicao=2122076&subst=0")

In [4]:
bsObje =  BeautifulSoup(emendas, "lxml")

In [5]:
pdfs = bsObje.findAll("a", {"class":"rightIconified iconDetalhe linkDownloadTeor"})

In [6]:
#pdfs

In [7]:
# Lista dos nomes de deputados

In [8]:
req = urlopen('http://www.camara.gov.br/proposicoesWeb/prop_emendas?idProposicao=2122076&subst=0')
soup = BeautifulSoup(req.read(), 'html.parser')

tables_ele = soup.findAll('tbody', {'class': 'coresAlternadas'})
deputados = []
for table_ele in tables_ele:
    for row in table_ele.findAll('tr'):
        cols = row.findAll('td')
        deputados.append(cols[3].text.strip())

In [9]:
conta = 0

In [11]:
# deputados

In [12]:
# Cria repositório final para guardar dados e cabeçalho

In [13]:
tabela_final = []

In [14]:
tabela_final.append(["Arquivo", "Deputado", "Autor", "Assunto", "Data_Criacao", "Data_Moficacao"])
tabela_final

[['Arquivo', 'Deputado', 'Autor', 'Assunto', 'Data_Criacao', 'Data_Moficacao']]

In [18]:
# Função para testar codecs
def str_decode(str):
    if str == None:
        return str
    else: 
        # Verifica qual o codec do sistema atual (codec "padrão")
        defaultcodec = sys.getdefaultencoding().lower()

        codec = chardet.detect(str)['encoding']

        if (defaultcodec != codec.lower()):
            return str.decode(codec) # Se o codec for diferente do sistema atual então decodifica
        else:
            return str # Se o codec for o do sistema atual então mantêm 

In [19]:
# Iteração com os links de PDFs
for name in pdfs:
    # Cria url
    url = name.get('href')
    url = "http://www.camara.gov.br/proposicoesWeb/" + url
    
    # Limpeza para acesso
    posicao = url.find('filename=') + 9
    arquivo = (url[posicao:])
    arquivo = arquivo.replace('+', ' ')
    arquivo = arquivo.replace('/', '-')
    arquivo = arquivo.replace('%3D%3E', '=-')
    arquivo = arquivo + ".pdf"
    
    # Baixa o arquivo
    urllib.request.urlretrieve(url, arquivo)
    
    # Extrai metadados
    fp = open(arquivo, 'rb')
    parser = PDFParser(fp)
    doc = PDFDocument(parser)
    dados_recuperados = doc.info[0]
    
    # Testa e grava campos 
    author = str_decode(dados_recuperados.get("Author"))
    subject = str_decode(dados_recuperados.get("Subject"))
    creation_date = str_decode(dados_recuperados.get("CreationDate"))
    mod_date = str_decode(dados_recuperados.get("ModDate"))
    nome_deputado = deputados[conta]
    conta = conta + 1
    
    # Grava uma linha completa
    tabela_final.append([arquivo, nome_deputado, author, subject, creation_date, mod_date])

In [20]:
# Grava o CSV
header = True
# Abre um arquivo no computador e o prepara para gravação
with open('emendas_autores.csv', 'a') as f:
    # usa a ferramenta "writer" do módulo csv
    writer = csv.writer(f)
    # se precisarmos gravar o cabeçalho...
    if header is True:
        # gravamos o cabeçalho
        writer.writerow(tabela_final[0])
        # marcamos que não é mais preciso gravar o cabeçalho
        header = False
    # gravamos o restante das linhas no CSV
    writer.writerows(tabela_final[1:])

In [21]:
# Lê o arquivo CSV

In [22]:
emd = pd.read_csv("emendas_autores.csv", sep = ',', encoding = 'latin_1')

In [23]:
emd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883 entries, 0 to 882
Data columns (total 6 columns):
Arquivo           883 non-null object
Deputado          883 non-null object
Autor             846 non-null object
Assunto           158 non-null object
Data_Criacao      883 non-null object
Data_Moficacao    850 non-null object
dtypes: object(6)
memory usage: 41.5+ KB


In [24]:
emd.head().reset_index()

,index,Arquivo,Deputado,Autor,Assunto,Data_Criacao,Data_Moficacao
0,0,EMC 1-2017 PL678716 =- PL 6787-2016.pdf,Laura Carneiro,Ivanete de Araujo Costa,EMD ADI - Emenda Aditiva,D:20170314114321-07'00',D:20170314114321-07'00'
1,1,EMC 2-2017 PL678716 =- PL 6787-2016.pdf,Laura Carneiro,Ivanete de Araujo Costa,EMD ADI - Emenda Aditiva,D:20170314114328-07'00',D:20170314114328-07'00'
2,2,EMC 3-2017 PL678716 =- PL 6787-2016.pdf,Laura Carneiro,Ivanete de Araujo Costa,EMD ADI - Emenda Aditiva,D:20170314114336-07'00',D:20170314114336-07'00'
3,3,EMC 4-2017 PL678716 =- PL 6787-2016.pdf,Laura Carneiro,MOD.CONLE.ST 2130/2017 - P_6736 - Davi Ribeiro...,EMD ADI - Emenda Aditiva,D:20170314114344-07'00',D:20170314114344-07'00'
4,4,EMC 5-2017 PL678716 =- PL 6787-2016.pdf,Laura Carneiro,Ivanete de Araujo Costa,EMD ADI - Emenda Aditiva,D:20170314114352-07'00',D:20170314114352-07'00'


In [25]:
emd.tail().reset_index()

,index,Arquivo,Deputado,Autor,Assunto,Data_Criacao,Data_Moficacao
0,878,EMP 28-2017 =- PL 6787-2016.pdf,Bebeto,NaN,NaN,D:20000101100657-03'00',NaN
1,879,EMP 29-2017 =- PL 6787-2016.pdf,Bebeto,NaN,NaN,D:20000101100710-03'00',NaN
2,880,EMP 30-2017 =- PL 6787-2016.pdf,Paulo Pereira da Silva,NaN,NaN,D:20000101083124-03'00',NaN
3,881,EMP 31-2017 =- PL 6787-2016.pdf,Renata Abreu,NaN,NaN,D:20000101090357-03'00',NaN
4,882,EMP 32-2017 =- PL 6787-2016.pdf,Evandro Gussi,NaN,NaN,D:20000101092046-03'00',NaN


Algumas análises

A ordem dos autores de emendas em número de redigidas (obs. em algumas esse campo foi deixado em branco)

In [26]:
emd.Autor.value_counts().reset_index()

,index,Autor
0,P_4189,99
1,Ricardo de Saboya Rocha Miranda,87
2,Dep. Jeronimo Goergen,47
3,Ivanete de Araujo Costa,42
4,Angelo Fabiano Farias da Costa,30
5,P_111067,27
6,P_6394,27
7,admin,25
8,Prodasen,25
9,Osmar de Oliveira Aguiar,21


Filtros pelos cinco primeiros principais autores, com respectivos deputados

In [27]:
top_autores = emd.Autor.value_counts().reset_index().head(5)

In [28]:
top_autores

,index,Autor
0,P_4189,99
1,Ricardo de Saboya Rocha Miranda,87
2,Dep. Jeronimo Goergen,47
3,Ivanete de Araujo Costa,42
4,Angelo Fabiano Farias da Costa,30


In [29]:
primeiro = emd[emd.Autor == "P_4189"]

In [30]:
primeiro.Deputado.value_counts().reset_index()

,index,Deputado
0,Mauro Lopes,22
1,Vanderlei Macris,20
2,Magda Mofatto,18
3,Gorete Pereira,11
4,CÃ©lio Silveira,10
5,Valdir Colatto,9
6,Arnaldo Jordy,8
7,Celso Maldaner,1


In [31]:
segundo = emd[emd.Autor == "Ricardo de Saboya Rocha Miranda"]

In [32]:
segundo.Deputado.value_counts().reset_index()

,index,Deputado
0,Major Olimpio,26
1,Diego Andrade,22
2,Renzo Braz,19
3,Lelo Coimbra,10
4,Valdir Colatto,10
